# Generate Convolutional Layers by Splatting Gaussians

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import glm
import math
import matplotlib.pyplot as plt
import diff_gaussian_rasterization_batched as dgrb

In [ ]:
device = torch.device("cuda")

In [ ]:
class GaussianSplattingLayer(nn.Module):
    @staticmethod
    def construct_renderer() -> dgrb.GaussianRasterizer:
        FoVx = 90.0
        FoVy = 90.0
        return dgrb.GaussianRasterizer(
            dgrb.GaussianRasterizationSettings(
            image_height=64,
            image_width=64,
            tanfovx=math.tan(FoVx * 0.5),
            tanfovy=math.tan(FoVy * 0.5),
            bg=torch.zeros(3, dtype=torch.float32, device=device),
            scale_modifier=1.0,
            viewmatrix=torch.eye(4, dtype=torch.float32, device=device),
            projmatrix=gen_orthographic_matrix(-1, 1, -1, 1, -1, 1).to(device),
            sh_degree=1,
            campos=torch.tensor([0, 0, 0], dtype=torch.float32, device=device),
            prefiltered=False,
            debug=False,
        )
        )
    
    
    def __init__(self, in_channels:int, out_channels:int, n_gaussians_per_layer:int)
        super(GaussianSplattingLayer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.n_gaussians_per_layer = n_gaussians_per_layer
        
        renderers = [construct_renderer() for _ in range(n_gaussians_per_layer)]